<a href="https://colab.research.google.com/github/perrin-isir/td_intro_classif_images/blob/main/td_classif_images.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>
<a id="raw-url" href="https://raw.githubusercontent.com/perrin-isir/td_intro_classif_images/main/td_classif_images.ipynb" download> <img align="left" src="https://img.shields.io/badge/Github-Download%20(Right%20click%20%2B%20Save%20link%20as...)-blue" alt="Download (Right click + Save link as)" title="Download Notebook"></a>

<center><u><b>TD : classification d'images et apprentissage profond</b></u></center>
<center>Nicolas Perrin-Gilbert, CNRS</center>

Dans ce TD, nous allons faire de la classification automatique d'images grâce à l'apprentissage profond et les réseaux de neurones convolutifs.

Ce fichier est un *notebook* [jupyter](https://fr.wikipedia.org/wiki/Jupyter). Utilisez les touches haut et bas pour naviguer de cellule en cellule, et appuyez sur *Maj+Entrée* ou *Ctrl+Entrée* pour exécuter une cellule. Vous pouvez aussi appuyer sur *Entrée* ou double-cliquer sur une cellule pour modifier son contenu. Si vous voulez recommencer depuis le début, sélectionnez l'onglet Kernel et cliquez sur Restart & Clear Output.

**IMPORTANT :** Ce notebook fonctionne mieux avec accélération GPU.  
Dans Colab : dans le menu Colab, choisissez Runtime > Change Runtime Type, puis sélectionnez 'GPU'.

Commençons par importer les modules python que nous utiliserons.
Il s'agit principalement de l'outil [TensorFlow](https://fr.wikipedia.org/wiki/TensorFlow) qui est un des logiciels les plus utilisés pour l'apprentissage profond. Allez sur la cellule suivante, et appuyez sur *Maj+Entrée*. Si tout se passe bien vous verrez s'afficher la phrase indiquant que les modules ont été chargés.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
import time
import random
import IPython
from IPython import get_ipython
print("C'est bon, les modules ont été chargés.")
print("tensorflow version:", tf.__version__)
print("- GPU:", tf.config.list_physical_devices('GPU'))
tf.random.set_seed(123456789)

Nous allons maintenant importer les données de la base [MNIST](https://fr.wikipedia.org/wiki/Base_de_donn%C3%A9es_MNIST), qui contient 70000 images de chiffres écrits à la main (60000 images pour l'apprentissage, et 10000 images de test) :

In [ ]:
import tensorflow.keras.datasets.mnist as mnist
(X, Y), (testX, testY) = mnist.load_data()
X = np.expand_dims(X, -1)/255.0
testX = np.expand_dims(testX, -1)/255.0

Les variables `X` et `testX` contiennent désormais des tableaux de valeurs, auxquelles on accède par exemple de la manière suivante :
    
    X[566, 7, 11, 0] 

La valeur ci-dessus est la valeur normalisée (entre 0 et 1 et non entre 0 et 255) du pixel de la 8ème rangée et de la 12ème colonne dans la 567ème image (décalage de 1 car les indices commencent à 0). Le dernier des 4 paramètres est toujours 0 car ce sont des images en noir et blanc (il y aurait trois valeurs par pixel pour des images en couleur). La valeur d'un pixel est un niveau de gris (0=blanc, 1=noir). `X` correspond aux images d'apprentissage, et `testX` aux images de test (qui ne sont pas utilisées pour entraîner le réseau de neurones, mais uniquement pour évaluer ses performances). Comme nous le verrons plus loin, il est important d'avoir des données de test séparées, non utilisées pendant l'apprentissage, car sinon l'algorithme pourrait ne donner de bons résultats que sur les données qu'il a déjà "vues", et très mal généraliser à de nouvelles données. Apprendre très bien sur les données vues sans être capable de généraliser correctement, cela s'appelle le *surapprentissage* ou *overfitting*, un phénomène très fréquent que les algorithmes d'apprentissage modernes essayent toujours d'éviter.

`Y` et `testY` contiennent des listes d'*étiquettes*, qui indiquent pour chaque image le chiffre qu'elle représente. Si la 567ème image représente un 6, alors `Y[566]` vaut 6, tout simplement. Comme pour `X` et `testX`, `Y` concerne les images d'apprentissage, et `testY` les images de test.

In [ ]:
print("X[566, 7, 11, 0] =", X[566, 7, 11, 0])

In [ ]:
print("Y[566] =", Y[566])

La dernière image des données d'apprentissage est un ...

In [ ]:
print("Y[59999] =", Y[59999])

... 8 !
Pour vérifier cela, il est utile de pouvoir visualiser des images. Exécutez la cellule suivante, qui contient la fonction `afficheImages` nous permettant de faire cela.

In [ ]:
def identite(x):
    return np.copy(x)

def afficheImages(l_in, xarray, fonction=identite):
    l = []
    for elt in l_in:
        l.append(fonction(xarray[elt:elt+1,:,:,:]))    
    cte = 30.0/9.0
    k = len(l)
    n = int(np.sqrt(cte * k))
    m = int(k/(n * 1.0))
    if (n*m<k):
        m = m+1
    width=20
    f, ax = plt.subplots(m,n,squeeze=False,figsize=(width,int(width*m/(n*2.0))))
    for i in range(m):
        for j in range(n):
            if(j+n*i < k):                
                ax[i,j].tick_params(axis=u'both', which=u'both',length=0)
                ax[i,j].set_xticks([])
                ax[i,j].set_yticks([])
                if k==1:
                    ax[i,j].set_xticks(np.arange(0.5,28.5,1), minor=True)
                    ax[i,j].set_yticks(np.arange(0.5,28.5,1), minor=True)
                    ax[i,j].grid(which='minor')
                ax[i,j].grid(False)
                ax[i,j].set_xticklabels([])
                ax[i,j].set_yticklabels([])
                if np.size(l[j+n*i],3)==1:   
                    ax[i,j].matshow(l[j+n*i][0,:,:,0], cmap='Greys', )
                else:
                    ax[i,j].imshow(l[j+n*i][0,:,:,:], )
            else:
                ax[i,j].axis('off')
    plt.show()

Affichons donc l'image n° 59999 :

In [ ]:
afficheImages([59999],X)

Il s'agit bien d'un 8 ! On voit que les images sont de taille 28 x 28 (pixels). Chaque image est donc un tableau (extrait de `X`) composé de 28 x 28 = 784 nombres tous compris entre 0 et 1. 

La fonction `afficheImages` permet d'afficher plusieurs images à la fois. Affichons par exemple les images n° 0, n° 566, n° 712, n° 801, et n° 918 :

In [ ]:
afficheImages([0,566,712,801,918],X)

Pour afficher des images des données de test, il suffit d'utiliser `testX` à la place de `X` :

In [ ]:
afficheImages([0,5,102,150,173,802,924,1068,5253,9999,342,1,536,865,23,645,858,327,233,834],testX)

Modifiez les indices pour afficher d'autres images :

In [ ]:
afficheImages([4,8,3,12,122,2444,2647,4789,6932,7999],testX)

Il désormais temps de construire des réseaux de neurones ! 
La fonction suivante `reseau1` utilise Keras, une interface populaire de TensorFlow, pour construire un réseau très simple avec 10 neurones en sortie. Il s'agit d'un réseau *entièrement connecté* sans couche cachée : il prend en entrée les valeurs des pixels de l'image, en reliant chaque pixel à chacun des 10 neurones de sortie.

In [ ]:
def reseau1(num_classes=10):
    model = tf.keras.Sequential(
        [
            tf.keras.Input(shape=(28,28,1)),
         
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],)
    return model

Voilà à quoi ressemble le réseau :

In [ ]:
if 'google.colab' in str(get_ipython()):
    img_src='<img src="https://raw.githubusercontent.com/perrin-isir/td_intro_classif_images/main/reseau.png">'
else:
    img_src='<img src="./reseau.png">'
IPython.display.HTML(img_src)

À chaque "fil" reliant un neurone d'entrée à un neurone de sortie correspond un *poids* (un nombre positif ou nul). Ces poids sont les paramètres du réseau, c'est ce qui change lorsque le réseau est entraîné. Comme il y a 10 fils et donc 10 poids pour chacun des 784 neurones en entrée, le nombre total de paramètres de ce réseau est 7840 (plus 10 paramètres de biais pour chaque neurone de sortie, chaque biais déterminant la valeur constante prise par un neurone lorsqu'il ne reçoit que des 0 en entrée).

La dernière couche de neurones, de taille 10, retourne un tableau (plus précisément un tenseur) de 10 nombres auquel on applique la fonction *softmax* qui le transforme en une sortie de 10 nombres, tous positifs, et dont la somme vaut 1, donc conformes à une distribution de probabilités. Si le troisième nombre de la sortie vaut 0.4, par exemple, cela signifie que la probabilité que l'image en entrée représente un 3 est estimée à 40%. Si tous les autres nombres de la sortie ont une valeur inférieure, alors on dit parfois que la *prédiction* du réseau est 3, ce qui est en réalité un léger abus du langage, puisqu'avec une probabilité de 40%, le réseau estime en réalité qu'il y a plus de chances que l'image en entrée représente autre chose qu'un 3.

L'objectif est d'entraîner le réseau pour que ses prédictions soient correctes.

Initialement, les poids sont déterminés aléatoirement.
L'entraînement du réseau fonctionne de la manière suivante : on choisit un petit ensemble d'images issues des données d'apprentissage (`X`) pour lesquelles on sait (grâce à `Y`) ce que le réseau devrait prédire. Cet ensemble d'images est appelé un *batch*. On regarde ce que le réseau prédit effectivement sur ce batch, et on détermine, grâce à un algorithme appelé la *rétropropagation du gradient*, comment modifier un tout petit peu les paramètres du réseau pour que les prédictions se rapprochent légèrement des prédictions correctes. Ensuite, on choisit un nouvel ensemble d'images, et on recommence la même opération. En réitérant cette étape un grand nombre de fois, les paramètres s'ajustent petit à petit et les prédictions du réseau deviennent de plus en plus correctes.

Instancions `reseau1` pour construire un réseau qui pourra être entraîné.

In [ ]:
reseau_1 = reseau1()

La fonction suivante, `afficheResultats`, permettra de visualiser les prédictions du réseau pour une liste d'images donnée en entrée de la même manière qu'avec la fonction `afficheImages`.

In [ ]:
def afficheResultats(model, l_in, xarray, fonction=identite):
    l = []
    for elt in l_in:
        l.append(fonction(xarray[elt:elt+1,:,:,:]))  
    cte = 30.0/9.0
    k = len(l)
    n = int(np.sqrt(cte * k))
    m = int(k/(n * 1.0))
    if (n*m<k):
        m = m+1
    width=20
    f, ax = plt.subplots(m,n,squeeze=False, figsize=(width,int(width*m/(n*2.0))))
    for i in range(m):
        for j in range(n):
            if(j+n*i < k):
                ax[i,j].tick_params(axis=u'both', which=u'both',length=0)
                ax[i,j].set_ylim([-0.5,10.5])
                ax[i,j].set_xlim([-1.5,12.5])
                ax[i,j].set_xticks([])
                ax[i,j].set_xticks(np.arange(0.5,9.5,1), minor=True)
                ax[i,j].set_yticks([])
                ax[i,j].grid(False)
                ax[i,j].set_aspect('equal')
                cm = plt.cm.get_cmap('RdYlBu_r')  
                L = model.predict(l[j+n*i][:,:,:,:], verbose=0)[0]
                C = [cm(x) for x in L]
                ax[i,j].barh(range(0,10), [z * 10.0 for z in reversed(L)], color=C)
                for idx in range(len(L)):
                    if L[idx]>0.02:
                        ax[i,j].text(10.0*L[idx]+0.15,len(L)-1-idx+0.1,idx)
            else:
                ax[i,j].axis('off')
    plt.show()

Voyons ce que le réseau prédit pour l'image n° 9999 de `testX` (qui représente un 6). Comme les poids sont au départ déterminés aléatoirement, on voit que le réseau donne à peu près la même valeur pour toutes les possibilités :  

In [ ]:
afficheResultats(reseau_1, [9999], testX)

Même avec des paramètres aléatoires, le réseau fait des prédictions, puisque chaque neurone de sortie ne renvoie pas exactement la même valeur. Grâce à la fonction `evaluate`, on peut évaluer la performance du réseau sur les images de test (`testX`). Cette fonction compte le nombre d'images pour lesquelles la prédiction du réseau est correcte, ainsi que le nombre d'images pour lesquelles le réseau se trompe, et elle renvoie le ratio entre ces deux nombres :

In [ ]:
reseau_1.evaluate(testX, testY)

On peut constater que le réseau effectue une prédiction correcte dans à peu près 10% des cas, ce qui est logique pour une prédiction aléatoire avec 10 choix possibles au total (et donc 9 chances sur 10 de se tromper).

Nous allons maintenant entraîner le réseau pendant un cycle (ou une *epoch*), c'est-à-dire que l'il va voir une fois l'ensemble des 60000 images d'apprentissage. Les *batchs* seront de taille 96 (il faudra 625 batchs pour effectuer le cycle entier, car 96 x 625 = 60000). C'est la fonction `fit()` qui nous permet d'effectuer cet apprentissage.

In [ ]:
start_time = time.time()
reseau_1.fit(X, Y, epochs=1, shuffle=True, validation_data=(testX, testY), 
                      batch_size=96)
print("Temps de calcul total : %.2f secondes" % (time.time() - start_time))

À la fin de l'entraînement, la performance du réseau est réévaluée sur les données de test, le résultat étant la valeur de `val_sparse_categorical_accuracy`.
On voit qu'après 1 cycle d'entraînement, le réseau s'est nettement amélioré et atteint une précision légèrement proche de 0.90, c'est-à-dire que dans plus de 9 cas sur 10 le réseau effectue une prédiction correcte.

Il est important de noter que les images de `testX` n'ont jamais été vues par le réseau pendant son entraînement, et pourtant il est capable de répondre correctement pour 90% d'entre elles.

Affichons à nouveau les résultats de la prédiction pour l'image n° 9999 :

In [ ]:
afficheResultats(reseau_1, [9999], testX)

De manière écrasante, le réseau prédit correctement que l'image représente un 6.

Modifiez les indices ci-dessous pour afficher les résultats pour d'autres images, et essayez de trouver des images pour lesquelles le réseau semble se tromper :

In [ ]:
afficheImages([0,5,102,150,173,802,924,1068,5253], testX)
afficheResultats(reseau_1, [0,5,102,150,173,802,924,1068,5253], testX)

Voici une liste d'images relativement ambigües, qui peuvent provoquer une "hésitation" du réseau (plusieurs neurones de sorties avec des valeurs assez hautes), et même celle d'un humain :

In [ ]:
afficheImages([33,62,358,1459,2780,4176, 5736, 7472,9505, 9754], testX)
afficheResultats(reseau_1, [33,62,358,1459,2780,4176, 5736, 7472,9505, 9754], testX)

`testY` permet de vérifier que les réponses correctes sur les images ci-dessus sont : 4, 9, 7, 2, 2, 2, 6, 2, 7, 5.

In [ ]:
for i in [33,62,358,1459,2780,4176,5736,7472,9505,9754]:
    print("testY[", i, "] =", testY[i])

On pourrait laisser le réseau apprendre sur un grand nombre de cycles (vous pouvez essayer !), mais avec l'architecture simple du réseau il n'est pas possible d'obtenir un fiabilité nettement supérieure à 92%.

Pour cela, il est nécessaire d'utiliser une architecture profonde qui contient beaucoup plus de paramètres. Voici un exemple de réseau profond :

In [ ]:
def reseau2(num_classes=10):
    model = tf.keras.Sequential(
        [
            tf.keras.Input(shape=(28,28,1)),
            tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],)
    return model

Voyons, sans rentrer dans les détails, quelles sont les différentes couches qui composent ce réseau.

La première couche est un réseau convolutif (`Conv2D(...)`) à 32 *features* et des *filtres* de taille 3 x 3. Dans cette couche, les neurones d'entrée sont organisés en une grille de dimensions 28 x 28 (comme les pixels de l'image), et les neurones de sortie sont organisés selon 3 dimensions : 28 x 28 en largeur et hauteur, et 32 en profondeur (ce qui correspond au nombre de *features*).

Dans chacune des 32 grilles de neurones de sortie, les neurones sont reliés à des neurones d'entrées qui ont à peu près la même position dans l'image. Plus précisément, un neurone de sortie est relié à un voisinage de taille 3 x 3 (la taille des filtres). 
Pour simplifier, supposons qu'il n'y ait non pas 32 mais 3 *features* :

In [ ]:
if 'google.colab' in str(get_ipython()):
    img_src='<img src="https://raw.githubusercontent.com/perrin-isir/td_intro_classif_images/main/convol.png">'
else:
    img_src='<img src="./convol.png">'
IPython.display.HTML(img_src)

Les neurones jaunes et rouges forment ce qu'on appelle les filtres, où chaque neurone dans les grilles en sortie est relié à des neurones de son voisinage dans les grilles en entrée. Ici l'entrée de ces filtres (qu'on appelle *champs récéptif*) est de taille 3 x 3 x 1 car il n'y a qu'une seule grille en entrée, mais les filtres ont toujours la même profondeur que l'entrée ; donc si l'entrée était composée de plusieurs grilles de neurones, par exemple 5, alors la taille des filtres serait 3 x 3 x 5. Cette règle fait qu'il n'est pas nécessaire de préciser la profondeur des filtres, on dit donc simplement que les filtres sont de taille 3 x 3. La sortie d'un filtre est toujours de taille 1 x 1 x *le nombre de features* (pour la couche de notre réseau 1 x 1 x 32, et sur le schéma ci-dessus 1 x 1 x 3).

Une particularité importante des réseaux convolutifs est que les filtres ont des poids **partagés**. Concrètement, cela veut dire que sur le schéma, les 9 x 3 "fils" du haut (filtre jaune) ont exactement les mêmes poids que les 27 "fils" du bas (filtre rouge).
Donc le nombre total de paramètres de toute la couche de convolution est en fait égal aux nombre de paramètres d'un seul filtre. Dans notre cas, avec 32 features, le nombre de paramètres est égal à 3 x 3 x 32 = 288 (plus 32 paramètres de biais).

La couche suivante est un *max-pooling* (`MaxPooling2D(...)`), qui dans chaque grille (de la sortie de la couche de convolution), regroupe les neurones en carrés de taille 2 x 2, et ne conserve que le neurone dont la valeur est la plus grande, comme ceci :

In [ ]:
if 'google.colab' in str(get_ipython()):
    img_src='<img src="https://raw.githubusercontent.com/perrin-isir/td_intro_classif_images/main/pooling.png">'
else:
    img_src='<img src="./pooling.png">'
IPython.display.HTML(img_src)

Cela a pour effet de diviser par deux les dimensions de la grille. 

Après le premier max-pooling suivent une nouvelle couche de convolution, un nouveau max-pooling, puis trois couches entièrement connectées. Voilà un schéma de l'architecture globale de notre réseau :

In [ ]:
if 'google.colab' in str(get_ipython()):
    img_src='<img src="https://raw.githubusercontent.com/perrin-isir/td_intro_classif_images/main/archi.png">'
else:
    img_src='<img src="./archi.png">'
IPython.display.HTML(img_src)

Remarque : le réseau utilise également une technique de *dropout* qui consiste à "éteindre" aléatoirement une certaine quantité de neurones à chaque pas d'apprentissage (50%). Cela rend les neurones moins dépendants les uns des autres, et empêche le *surapprentissage* (qui se traduit par une très grande variabilité et de mauvais résultats sur des données nouvelles).

En tout, le réseau global a 423178 paramètres à entraîner.

In [ ]:
reseau_2 = reseau2()
# reseau_2.summary()

Entraînons le réseau sur un cycle :

In [ ]:
start_time = time.time()
reseau_2.fit(X, Y, epochs=1, shuffle=True, validation_data=(testX, testY), 
                      batch_size=96)
print("Temps de calcul total : %.2f secondes" % (time.time() - start_time))

Le réseau ayant environ 50 fois plus de paramètres, chaque pas d'apprentissage prend plus de temps, et donc un cycle total prend également nettement plus de temps. Mais on remarque tout de même qu'avec 1 seul cycle, le réseau atteint déjà une précision supérieure à 95%, ce qui est inatteignable avec le premier réseau. En entraînant le réseau sur un grand nombre de cycle, il est facile de dépasser les 98% de fiabilité.

Cet écart de performance entre les 2 types d'architectures devient encore plus net lorsque les images et les catégories deviennent plus complexes. Pour cela, chargeons une base d'image plus complexe que MNIST, appelée CIFAR-10. Il s'agit de 60000 petites images en couleur d'avions, voitures, oiseaux, chats, cerfs, chiens, grenouilles, chevaux, bateaux et camions, toutes étiquetées comme appartenant à l'une de ces 10 catégories. Il y a 50000 images d'apprentissage et 10000 images de test.

In [ ]:
import tensorflow.keras.datasets.cifar10 as cifar10
(X2, Y2), (testX2, testY2) = cifar10.load_data()
X2 = X2/255.0
testX2 = testX2/255.0

Affichons quelques-unes de ces images (de taille 32 x 32) :

In [ ]:
afficheImages([19,28,101,247,999,1506,2229,4010,5488,7702,7778,
               8674,11898,13647,20056,22801,28994,35000,37373,39001, 
               41571,44266,48765,49999], X2)

Commençons par créer la même structure de réseau très simple que `reseau1`, mais adaptée aux images de CIFAR-10 (c'est-à-dire de taille 32 x 32 et avec 3 nombres par pixels (pour les couleurs) :

In [ ]:
def reseau3(num_classes=10):
    model = tf.keras.Sequential(
        [
            tf.keras.Input(shape=(32,32,3)),
         
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],)
    return model

In [ ]:
reseau_3 = reseau3()

Entraînons le réseau :

In [ ]:
reseau_3.fit(X2, Y2, epochs=1, shuffle=True, validation_data=(testX2, testY2), batch_size=96)

Le score est nettement moins bon que sur MNIST : à peine 35% de fiabilité après l'entraînement sur un cycle. 
Et même en essayant d'entraîner le réseau sur plus de cycles, on s'aperçoit qu'il est très difficile de faire mieux que 40%. En réalité, la structure simple du réseau n'est pas capable de classifier efficacement les différents types d'images, quelle que soit la durée de l'apprentissage. Reprenons donc une architecture profonde, similaire à celle du `reseau2` : 

In [ ]:
def reseau4(num_classes=10):
    model = tf.keras.Sequential(
        [
            tf.keras.Input(shape=(32,32,3)),
            tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],)
    return model

In [ ]:
reseau_4 = reseau4()

In [ ]:
reseau_4.fit(X2, Y2, epochs=1, shuffle=True, validation_data=(testX2, testY2), batch_size=96)

Après l'entraînement sur un cycle, on voit que la performance est déjà d'environ 50%. 
Essayons avec une architecture encore plus profonde (3 couches de convolution et des variantes au niveau de l'utilisation du max-pooling et du dropout) :

In [ ]:
def reseau5(num_classes=10):
    model = tf.keras.Sequential(
        [
            tf.keras.Input(shape=(32,32,3)),
            tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(512, activation="relu"),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],)
    return model

In [ ]:
reseau_5 = reseau5()

In [ ]:
reseau_5.fit(X2, Y2, epochs=1, shuffle=True, validation_data=(testX2, testY2), batch_size=96)

Cette fois-ci, on obtient une fiabilité de plus de 50% dès le premier cycle. Mais ce qui est vraiment important, c'est qu'en entraînant le réseau sur beaucoup plus de cycles, la performance continue d'augmenter. Nous allons charger l'état du réseau après un entraînement sur 50 cycles (entraînement qui peut prendre un certain temps, selon la machine dont on dispose) :

In [ ]:
if 'google.colab' in str(get_ipython()):
    tmp_dir = os.path.join(os.path.expanduser("~"), "tmp_data")
    !wget -P {tmp_dir} "https://raw.githubusercontent.com/perrin-isir/td_intro_classif_images/main/net5.keras"
    reseau_5 = tf.keras.models.load_model(os.path.join(os.path.expanduser("~"), "tmp_data", "net5.keras"))
else:
    reseau_5 = tf.keras.models.load_model("net5.keras")

Remarque : pour sauvegarder l'état du réseau il suffit de faire :
    
    reseau_5.save("net5.keras")
        
La fiabilité du réseau est d'un peu moins de 80% :

In [ ]:
print(reseau_5.evaluate(testX2, testY2))

Il est intéressant de remarquer que la fiabilité est nettement plus élevée si on l'évalue sur les données qui ont été vues durant l'apprentissage (ici on le fait sur les 10000 premières images de la base de donnée d'apprentissage) :

In [ ]:
print(reseau_5.evaluate(X2[0:10000,:,:,:], Y2[0:10000]))

Tout l'intérêt de l'apprentissage est dans la généralisation, et il ne sert à rien d'évaluer un réseau sur les données d'apprentissage, surtout si le réseau a un très grand nombre de paramètres (ce qui lui permet en quelques sortes de se "souvenir" des résultats pour toutes les données d'apprentissage). Le fait de tester le réseau sur des données non utilisées pendant l'apprentissage s'appelle la *cross-validation*.

Lorsque la performance est très différente sur les images d'apprentissage et sur les images de test, on dit que le réseau fait du *surapprentissage* (ou de l'*overfitting*).

Voici quelques résultats sur les données de test :

In [ ]:
afficheImages([79,128,201,344,578,648,776,1090,1201,1257,1305,1511,3291,4137,5072,7144,7458,8775,9999], testX2)
afficheResultats(reseau_5,[79,128,201,344,578,648,776,1090,1201,1257,1305,1511,3291,4137,5072,7144,7458,8775,9999], testX2)
print("0=avion, 1=voiture, 2=oiseau, 3=chat, 4=cerf, 5=chien, 6=grenouille, 7=cheval, 8=bateau, 9=camion")

80% de réussite est loin d'être parfait, mais cela permet tout de même d'obtenir quelques prédictions intéressantes.
Essayez de trouver des images de test pour lesquelles le réseau renvoie des résultats faux ou ambigus (plusieurs valeurs hautes).

Pour obtenir d'encore meilleurs résultats et réduire l'overfitting, on peut appliquer un *preprocessing* (pré-traitement) aux images, par example en les centrant ou en normalisant leurs valeurs, ainsi qu'une étape de *data augmentation* qui consiste à légèrement modifier les images en entrée par des transformations aléatoires (rotations, décalages, cisaillements, zooms, retournement, ...), afin d'augmenter artificiellement le volume de données d'entraînement. Voici comment préparer pre-processing et data augmentation :

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

data_augment_train = datagen.flow(X2, Y2, batch_size=96)

Voilà le type d'images que cela génère:

In [ ]:
for batch in datagen.flow(X2, batch_size=20):
    break
afficheImages(np.arange(batch.shape[0]), batch)

Ensuite, pour entraîner le réseau :
    
    reseau_5.fit(data_augment_train, epochs=50, validation_data=(testX2, testY2))

In [ ]:
# reseau_5.fit(data_augment_train, epochs=50, validation_data=(testX2, testY2))
# reseau_5.save("net5_augmented.keras")
reseau_5 = tf.keras.models.load_model("net5_augmented.keras")

Évaluation des performances du réseau sur les données de test :

In [ ]:
print(reseau_5.evaluate(testX2, testY2))

Les meilleurs réseaux sont capables d'atteindre un score supérieur à 95%, ce qui est légèrement supérieur au score moyen d'un humain. Mais même avec des scores comparables à ceux des humains, l'apprentissage des réseaux de neurones est très différent de celui des humains. On s'en aperçoit de manière flagrante lorsqu'on demande de la flexibilité et de la robustesse (ce pour quoi les humains sont très forts). 

Par exemple, voici une fonction *efface* qui met à 0 les valeurs de 4% des pixels d'une image :

In [ ]:
def efface(x):
    z = np.copy(x)
    len_x = np.shape(x)[1]
    len_y = np.shape(x)[2]
    for i in range(len_x):
        for j in range(len_y):
            rn = random.randint(1,25)
            if(rn == 1):
                z[:,i,j,:] = 0
    return z

La fonction `efface` peut être passée en argument aux fonctions `afficheImages` et `afficheResultats` qui vont l'appliquer aux images avant de les afficher ou d'évaluer les prédictions du réseau. Voici les résultats sur quelques images :

In [ ]:
afficheImages([79,128,201,344,578,648,776,1201,1257,1511,3291,5072,7144,7458,9999], testX2, efface)
print("Résultats sans la fonction efface :")
afficheResultats(reseau_5,[79,128,201,344,578,648,776,1201,1257,1511,3291,5072,7144,7458,9999], testX2)
print("Résultats avec la fonction efface :")
afficheResultats(reseau_5,[79,128,201,344,578,648,776,1201,1257,1511,3291,5072,7144,7458,9999], testX2, efface)
print("0=avion, 1=voiture, 2=oiseau, 3=chat, 4=cerf, 5=chien, 6=grenouille, 7=cheval, 8=bateau, 9=camion")

Pour les humains, les quelques pixels en moins ne changent en général pas la prédiction (qu'elle soit bonne ou mauvaise). Mais on voit que pour le réseau cela suffit à changer complètement certains résultats.

Voilà, nous arrivons à la fin du TD, dans lequel nous avons vu comment utiliser des réseaux de neurones profonds pour apprendre à reconnaître des images, à conditions bien sûr que l'on dispose d'une grande base de données étiquetées. Nous avons vu la structure habituelle de ces réseaux profonds, et nous avons vu que pour des images relativement complexes, la profondeur du réseau est importante pour obtenir de bons résultats.

Voici quelques défis de programmation pour aller plus loin :
* **Défi 1 **: écrivez quelques fonctions qui vous permettront d'évaluer facilement votre propre score sur des images prises au hasard dans les données de test, et de comparer vos résultats avec ceux du `reseau_5` sur les mêmes images.

* **Défi 2 **: écrivez un algorithme qui cherche, dans la base d'images de test, un chien qui, selon le `reseau_5`, ressemble à un cheval (ou une voiture qui ressemble à un bateau, etc.).

* **Défi 3 **: en reprenant la fonction `efface` comme point de départ, essayez d'écrire une fonction qui met dans les images des modifications presque invisibles (ou au moins pas du tout gênantes) pour les humains, mais qui trompent complètement le `reseau_5`.

Remarque : pour les défis, la fonction suivante `predict` pourra être utile. 

In [ ]:
def predict(model, l_in, xarray, fonction=identite):
    l = []
    L = []
    for elt in l_in:
        l.append(fonction(xarray[elt:elt+1,:,:,:]))     
    for i in range(len(l)):
        L.append(np.argmax(model.predict(l[i][:,:,:,:], verbose=0)[0]))
    return L

Cette fonction permet de calculer la prédiction d'un réseau sur des images. Par exemple, voici les prédictions de `reseau_5` sur les images n° 344, n° 1701 et n° 3000 de `X2` :

In [ ]:
print(predict(reseau_5, [344, 547, 2999], X2))
print("0=avion, 1=voiture, 2=oiseau, 3=chat, 4=cerf, 5=chien, 6=grenouille, 7=cheval, 8=bateau, 9=camion")

On vérifie que le réseau ne se trompe pas :

In [ ]:
afficheImages([344, 547, 2999], X2)